In [237]:
dataset_name = "arguana"

In [238]:
DATA_DIR = f"/home/jinho/FlagEmbedding_Aizip/dataset/{dataset_name}"

In [239]:
import json

with open(f'{DATA_DIR}/corpus.jsonl', 'r') as f:
    corpus = [json.loads(line) for line in f]

In [240]:
with open(f'{DATA_DIR}/test_queries.jsonl', 'r') as f:
    queries = [json.loads(line) for line in f]

In [241]:
with open(f'{DATA_DIR}/test_qrels.jsonl', 'r') as f:
    qrels = [json.loads(line) for line in f]

In [242]:
import os
import re

In [245]:
path = f"/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/{dataset_name}"
print("Select the output path from below")
for entry in os.listdir(path):
    print(os.path.join(path, entry))

Select the output path from below
/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/arguana/20250226_0347


In [249]:
pattern = r"_(\d+\.\d+)_rank_(\d+)"

path = "/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/arguana/20250226_0347"

eval_metric = "recall_at_3"

top_metric = 0
top_path = None

for entry in os.listdir(path):
    match = re.search(pattern, entry)
    alpha, rank = match.groups()
    full_path = os.path.join(path, entry)
    if os.path.isdir(full_path):
        embedder_path = os.path.join(full_path, os.listdir(full_path)[0])
        for ranker in os.listdir(embedder_path):
            if ranker != "NoReranker":
                embedder_ranker_path = os.path.join(embedder_path, ranker)
                eval_path = os.path.join(embedder_ranker_path, "EVAL/eval_results.json")
                with open(eval_path, "r") as f:
                    eval = json.load(f)
                    metric = eval[list(eval.keys())[0]][eval_metric]
                    if metric > top_metric:
                        top_metric = metric
                        top_path = embedder_ranker_path

In [250]:
top_path

'/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/arguana/20250226_0347/alpha_0.9_rank_3/bge-large-en-v1.5/bge-reranker-large'

In [251]:
top_metric

0.69559

In [252]:
for file_name in os.listdir(top_path):
    if "json" in file_name:
        search_result_fpath = os.path.join(top_path, file_name)

In [253]:
search_result_fpath

'/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/arguana/20250226_0347/alpha_0.9_rank_3/bge-large-en-v1.5/bge-reranker-large/arguana-test.json'

In [254]:
with open(search_result_fpath, "r") as f:
    search_results = json.load(f)

In [255]:
results = search_results['search_results']

In [256]:
corpus_dic = {}
for c in corpus:
    if c['id'] in corpus_dic:
        print(c['id'])
    corpus_dic[c['id']] = c['text']

In [257]:
qrel_dic = {}
for d in qrels:
    qid = d['qid']
    docid = d['docid']
    if docid not in corpus_dic:
        print(docid)
    if d['relevance'] != 0:
        if qid not in qrel_dic:
            qrel_dic[qid] = [docid]
        else:
            # print("exists")
            qrel_dic[qid].append(docid)

test-free-speech-debate-yfsdfkhbwu-con03b
test-education-ufsdfkhbwu-con03b
test-politics-dhwem-pro06b
test-science-sghwbdgmo-con03b
test-society-asfhwapg-con04b


In [258]:
len(results)

1406

In [259]:
len(corpus)

8674

In [260]:
len(qrel_dic)

1406

In [261]:
correct = []
incorrect = []
for q in queries:
    qid = q['id']
    qtext = q['text']
    gt_corpus_list = qrel_dic[qid]
    retrieved_corpus = list(results[qid].keys())
    found = False
    for gt_corpus in gt_corpus_list:
        if gt_corpus in retrieved_corpus:
            correct.append(qid)
        else:
            context = ""
            for i in range(len(retrieved_corpus)):
                c = retrieved_corpus[i]
                context += f"CONTEXT {i}\n\n"
                context += corpus_dic[c]
                context += "\n\n"
            if gt_corpus not in corpus_dic:
                c = "corpus not included (most likely a bug)"
            else:
                c = corpus_dic[gt_corpus]
            incorrect.append({"input": qtext, "gt_corpus": c, "retrieved": context})

In [262]:
print(incorrect[0]['retrieved'])

CONTEXT 0

There is a great moral difference between humans and animals. Unlike animals, humans are capable of rational thought and can alter the world around them. Other creatures were put on this earth for mankind to use, and that includes eating meat. For all these reasons we say that men and women have rights and that animals don’t. This means that eating meat is in no way like murder. It is natural for human beings to farm, kill, and eat other species. In the wild there is a brutal struggle for existence. The fact that we humans have succeeded in that struggle by exploiting our natural environment means that we have a natural right over lower species. In fact farming animals is much less brutal than the pain and hardship that animals inflict on each other naturally in the wild.  Eating meat does not need to mean cruelty to animals. There are a growing number of organic and free-range farms that can provide meat without cruelty to animals. Similarly, it might be reasonable to argue

In [263]:
len(correct)

978

In [264]:
len(correct) + len(incorrect)

1406

In [265]:
len(correct) / (len(correct) + len(incorrect))

0.6955903271692745

In [266]:
print(top_metric)

0.69559


In [267]:
import csv

In [268]:
import os

In [269]:
dirname = os.path.dirname(search_result_fpath)

In [270]:
dirname

'/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/arguana/20250226_0347/alpha_0.9_rank_3/bge-large-en-v1.5/bge-reranker-large'

In [273]:
# Write to CSV file
with open(f'{dirname}/{dataset_name}_missed_retrieval.csv', 'w', newline='') as csvfile:
    # Get the field names from the first dictionary
    fieldnames = incorrect[0].keys()
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write header
    writer.writeheader()
    
    # Write rows
    writer.writerows(incorrect)

print("CSV file written successfully!")

print(f'{dirname}/{dataset_name}_missed_retrieval.csv')

CSV file written successfully!
/home/jinho/FlagEmbedding_Aizip/beir/aizip/search_result/arguana/20250226_0347/alpha_0.9_rank_3/bge-large-en-v1.5/bge-reranker-large/arguana_missed_retrieval.csv
